In [6]:
# loaded_model = pickle.load("squeezenetfinetuned.pkl")
import sklearn 
import numpy as np
import joblib
import pickle
import torch

#Basma
# classifer = joblib.load("C:/Users/basma/Fourth Year/Graduation_Project/squeezenetfinetuned.pkl")

#Bassem
# classifer = joblib.load("squeezenetuntrained.pkl")

model = pickle.load(open('new squeezenet(finetuned).pkl', 'rb'))
model.eval()

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

In [9]:
# # Segmentation then Cropping algorithm 
# import cv2
# from imutils import contours
# from skimage.filters import threshold_otsu

# # Load image, grayscale, Otsu's threshold
# image_array = []
# results = []
# # image = cv2.imread('C:/Users/basma/Fourth Year/Graduation_Project/O001 (1).png')
# # image_array.append(image)
# # gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# # thresh = cv2.threshold(gray,0,255,cv2.THRESH_OTSU + cv2.THRESH_BINARY)[1]
# # inv = cv2.bitwise_not(thresh)
# # rgb = cv2.cvtColor(inv, cv2.COLOR_GRAY2RGB)

# # Load image
# image = cv2.imread('walk (1).png')

# # Convert image to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # Threshold image
# _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

# # Find contours
# contours,_  = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


# #---------------------------------------------------------------------
# # def Otsu(myImage):

#   # image = cv2.imread(myImage)
#   # gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#   # thresh = cv2.threshold(gray,0,255,cv2.THRESH_OTSU + cv2.THRESH_BINARY)[1]
#   # inv = cv2.bitwise_not(thresh)
#   # rgb = cv2.cvtColor(inv, cv2.COLOR_GRAY2RGB)
#   # # image_array.append(rgb)

#   # return inv








#   # sample_image = cv2.imread(myImage)
#   # img = cv2.cvtColor(sample_image,cv2.COLOR_BGR2RGB)

#   # # plt.axis('off')
#   # # plt.imshow(img)

#   # img_gray=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

#   # thresh = threshold_otsu(img_gray)
#   # img_otsu  = img_gray < thresh

#   # # plt.imshow(img_otsu)

#   # filtered = filter_image(img, img_otsu)
#   # filteredBW=cv2.cvtColor(filtered,cv2.COLOR_RGB2GRAY)
#   # return filteredBW



# def filter_image(image, mask):

#     r = image[:,:,0] * mask
#     g = image[:,:,1] * mask
#     b = image[:,:,2] * mask

#     return np.dstack([r,g,b])

# # image = Otsu('D:/semester 7/Graduation Project/Datasets/new dataset/O001.png')

# # gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# # thresh = cv2.threshold(gray,0,255,cv2.THRESH_OTSU + cv2.THRESH_BINARY)[1]
# # inv = cv2.bitwise_not(thresh)

# cv2.imwrite('image2.png', image)
# #---------------------------------------------------------------------

# # Find contours, sort from left-to-right, then crop
# # cnts = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# # cnts = cnts[0] if len(cnts) == 2 else cnts[1]
# # cnts, _ = contours.sort_contours(cnts, method="left-to-right")

# # ROI_number = 0
# # for c in cnts:
# #     area = cv2.contourArea(c)
# #     if area > 50:
# #         x,y,w,h = cv2.boundingRect(c)
# #         ROI = 255 - image[y:y+h, x:x+w]
# #         # img1 = cv2.copyMakeBorder(ROI, 20,20,20,20,cv2.BORDER_CONSTANT, value=[255,255,255])
# #         img2 = cv2.bitwise_not(ROI)
# #         cv2.imwrite('ROIVVV_{}.png'.format(ROI_number), img2)
        
# #         image_array.append(img2)
# #         cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 2)
# #         ROI_number += 1










# # Crop symbols
# for i, contour in enumerate(contours):
#     # Get bounding rectangle
#     x, y, w, h = cv2.boundingRect(contour)

#     # Crop symbol
#     symbol = image[y:y+h, x:x+w]

#     # image = cv2.imread(symbol)
#     gray = cv2.cvtColor(symbol, cv2.COLOR_BGR2GRAY)
#     thresh = cv2.threshold(gray,0,255,cv2.THRESH_OTSU + cv2.THRESH_BINARY)[1]
#     inv = cv2.bitwise_not(thresh)

#     # Save symbol
#     cv2.imwrite(f"symbol{i}.png", inv)





In [7]:
import cv2
import numpy as np

deleteCounterImage = 0

# Set threshold values for binary image and blob analysis
binary_threshold = 50
blob_min_area = 100

# Load the input image
image = cv2.imread("experiments/call out.png")

# Convert image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Otsu's thresholding method to get a binary image
_, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Invert the binary image
binary = cv2.bitwise_not(threshold)

# Perform blob analysis
contours, hierarchy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Sort contours by x-coordinate and then by y-coordinate
# contours = sorted(contours, key=lambda c: (cv2.boundingRect(c)[0], cv2.boundingRect(c)[1]))
# Sort contours from top to bottom and left to right
contours = sorted(contours, key=lambda c: (cv2.boundingRect(c)[1], cv2.boundingRect(c)[0]))

# Iterate over the sorted contours
for i, contour in enumerate(contours):
    
    # Check if the area of the contour is large enough to be considered an object
    if cv2.contourArea(contour) > blob_min_area:
        # Get bounding box coordinates
        x, y, w, h = cv2.boundingRect(contour)

        # Crop the image using the bounding box coordinates
        crop_img = image[y:y+h, x:x+w]
        # Save the cropped image to a file
        filename = f"crop_{i}.png"
        cv2.imwrite(filename, crop_img)
        

In [8]:
from PIL import Image, ImageFilter
import glob

image_array = []
image_files = glob.glob("*crop*.png")
for i, image_file in enumerate(image_files):
    
    print("ana: ",i)
    # Load the input image
    image1 = cv2.imread(image_file)

    # Get the original dimensions of the image
    height, width, _ = image1.shape

    # Calculate the aspect ratio of the image
    aspect_ratio = width / height

    # Set the desired output size (height or width)
    output_size = 224

    # Calculate the other dimension based on the aspect ratio
    if width > height:
        new_width = output_size
        new_height = int(new_width / aspect_ratio)
    else:
        new_height = output_size
        new_width = int(new_height * aspect_ratio)

    # Resize the image
    resized_image = cv2.resize(image1, (new_width, new_height))

    # Calculate the coordinates to copy the resized image onto the new image
    x_offset = int((output_size - new_width) / 2)
    y_offset = int((output_size - new_height) / 2)

    # Save the resized image to a file
    cv2.imwrite(f"resize{i}.png", resized_image)

    # Load the resized image from file
    resized_image = cv2.imread(f"resize{i}.png")

    # Convert image to grayscale
    gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    # Apply Otsu's thresholding method to get a binary image
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY)[1]
    # Invert the binary image
    inv = cv2.bitwise_not(thresh)
    # Save the segmented image to a file
    cv2.imwrite(f"seg_img{i}.png", inv)
    # Load the segmented image from file
    seg_img = cv2.imread(f"seg_img{i}.png")

    # Create a new blank image with the desired output size
    new_image = np.zeros((output_size, output_size, 3), np.uint8)
    new_image[:, :] = (0, 0, 0)  # fill with black pixels

    # Copy the segmented image onto the new image with black pixels
    new_image[y_offset:y_offset+new_height, x_offset:x_offset+new_width] = seg_img

    # Save the new image to a file
    cv2.imwrite(f"final{i}.png", new_image)
    deleteCounterImage = deleteCounterImage+1
    image_array.append(new_image)


ana:  0
ana:  1
ana:  2
ana:  3


In [11]:
import torchvision.transforms as transforms
from PIL import Image
from PIL import ImageFile
import cv2

results = ""
#Basma
for i in range(deleteCounterImage):
    print(deleteCounterImage)
    input_image = Image.open(f'final{i}.png').convert('RGB')
    # input_image = Image.open('C:/Users/basma/Fourth Year/Graduation_Project/SegmentedDatasetAgain/train/A040/A040_1.png').convert('RGB')
    #Bassem
    # input_image = Image.open('D:/Ahmed Bassem/MIU/Year 4/Graduation Project/Datasets/SegmentedDataset/SegmentedDataset/test/O001/O001_1.png').convert('RGB')

    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        # transforms.Pad(50),
    ])
    input_tensor = preprocess(input_image)
    # input_tensor = preprocess(Image.fromarray(i))
    input_batch = input_tensor.unsqueeze(0)
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch)

    # Tensor of shape 1000, with confidence scores over Aegyptos' 1072 classes
    print(output[0])
    # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
    probabilities = torch.nn.functional.softmax(output[0], dim=0)

    # Read the categories
    #Basma
    with open("classes_names.txt", "r") as f:

    #Bassem
    # with open("D:/Ahmed Bassem/MIU\Year 4/Graduation Project/classes_names.txt", "r") as f:
        categories = [s.strip() for s in f.readlines()]
    # Show top categories per image
    top5_prob, top5_catid = torch.topk(probabilities, 5)
    for i in range(top5_prob.size(0)):
        print(categories[top5_catid[i]], top5_prob[i].item())

    top1 = categories[top5_catid[0]]
    print(top1)

    class_name = 'T035'
    class_index = categories.index(class_name)
    class_prob = probabilities[class_index] * 100
    print(f"Percentage of confidence for {class_name}: {class_prob:.2f}%")

    # results = results + top1 + " "
    results += " "+ top1
updated_results = results.replace(" ", "", 1)
    # results.append(top1)
print(updated_results)

4
tensor([10.7141, 20.1104, 18.8000,  ..., 17.3920, 13.1781, 11.7310])
N035 0.9998297691345215
F032 8.666761277709156e-05
N035A 2.9037102649454027e-05
O043 2.4112581741064787e-05
E002 2.2730964701622725e-05
N035
Percentage of confidence for T035: 0.00%
4
tensor([12.2957,  5.5214,  3.6229,  ...,  2.0676,  2.2649,  4.9385])
U036 0.8414551019668579
T035 0.08839047700166702
S002 0.056926336139440536
AA007B 0.002605242421850562
S001 0.0010968808783218265
U036
Percentage of confidence for T035: 8.84%
4
tensor([7.9133, 7.4948, 4.5658,  ..., 8.9015, 4.7452, 8.5865])
S029 0.5291972160339355
V011C 0.12817814946174622
M022A 0.12496911734342575
S039 0.0745176449418068
M004 0.06928039342164993
S029
Percentage of confidence for T035: 0.00%
4
tensor([35.5414, 59.6606, 42.7982,  ..., 23.9993, 18.8984, 20.9682])
A002 0.9997575879096985
A005 0.00015853656805120409
A009 5.323188815964386e-05
D050F 8.782612894719932e-06
A051 3.454467787378235e-06
A002
Percentage of confidence for T035: 0.00%
N035 U036 S02

In [10]:
import matching_algorithm as ma
print(ma.dictionary_matching(updated_results))


[(['T035 S029 N035 A002'], 89, 'Call out'), (['A021 S029 G017 T034 N035 AA001'], 65, 'Friend'), (['E034 N035 G017 A002'], 63, 'Eat'), (['A021 U033 D033 N035'], 63, 'Statue'), (['A025 N023 N035 Z001'], 63, 'Utterance, recitation')]
(['T035 S029 N035 A002'], 89, 'Call out')
temp =N035 U036 S029 A002
result False
answer -
Call out


In [5]:
import os

# Delete previously created images
cropImageRemovals = glob.glob("*crop*.png")
for i in range(deleteCounterImage):
    removeCrop = cropImageRemovals
    removeSegImg = f"seg_img{i}.png"
    removeFinal = f"final{i}.png"
    removeResize = f"resize{i}.png"
    try:
        os.remove(removeCrop[i])
        os.remove(removeSegImg)
        os.remove(removeFinal)
        os.remove(removeResize)
    except FileNotFoundError:
        pass

In [8]:
import matching_algorithm_pronunciation as ma
print(ma.dictionary_matching_pronunciation(updated_results))

بري


In [9]:
import googletrans
from googletrans import *

In [10]:
match = ma.dictionary_matching(updated_results)

AttributeError: module 'matching_algorithm_pronunciation' has no attribute 'dictionary_matching'

In [17]:
translator = googletrans.Translator()
translate=translator.translate(match,dest='arabic')
print(translate.text)

يمشي
